# Analyze adsorption isotherm anomalies

In [24]:
import json
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import pprint
import os
import pygaps
import scipy

## Lets make a Python class for an adsorption isotherm

### This is usually referred to as class-based Object-Oriented Programming (OOP). 

#### [Here](https://pythonprogramming.net/introduction-learn-python-3-tutorials/)'s a tutorial series on Python if you want to learn the basics, and [Here](https://pythonprogramming.net/introduction-intermediate-python-tutorial/)'s an intermediate Python tutorial if the basic one is too easy.

#### More specifically, [This](https://pythonprogramming.net/object-oriented-programming-introduction-intermediate-python-tutorial/) is a tutorial on OOP in Python that I recommend, which introduces Classes

In [34]:
class Isotherm:
    # In Python classes, there is one required function you have to define, __init__ (for initialization)
    def __init__(self, filename):
        # This if-else sentence loads in the isotherm data. If you have extracted the isotherms into your directory, 
        #  you will load it directly, but if you haven't you'll have to download it from the NIST database.
        if os.path.isfile('isotherms/' + filename + '.json'):
            with open('isotherms/' + filename + '.json') as infile:
                isotherm_data = json.load(infile)
        else:
            url = "https://adsorption.nist.gov/isodb/api/isotherm/" + filename + ".json"
            isotherm_data = json.loads(requests.get(url).content)

        
        self.filename = filename
        self.doi = isotherm_data['DOI']
        assert len(isotherm_data['adsorbates']) < 2
        self.adsorbate = isotherm_data['adsorbates'][0]['name']
        self.inchikey = isotherm_data['adsorbates'][0]['InChIKey']
        self.adsorbent = isotherm_data['adsorbent']['name']
        self.category = isotherm_data['category']
        self.temperature = isotherm_data['temperature']
        self.adsorptionUnits = isotherm_data['adsorptionUnits']
        self.pressureUnits = isotherm_data['pressureUnits']
        self.pressure = np.array([isotherm_data['isotherm_data'][i]['pressure'] for i in range(len(isotherm_data['isotherm_data']))])
        self.adsorption = np.array([isotherm_data['isotherm_data'][i]['species_data'][0]['adsorption'] for i in range(len(isotherm_data['isotherm_data']))])
        self.henry_constant = None
        
    # Lets define a few helper functions first
    def _get_formula_from_inchikey(self):
        # Usually if there's an underscore in front of a function name, 
        #  it's meant for internal use
        url = "https://adsorbents.nist.gov/isodb/api/gas/" + self.inchikey + ".json"
        gas_data = json.loads(requests.get(url).content)
        self.formula = gas_data['formula']
    
    def get_molecular_weight(self):
        self._get_formula_from_inchikey()
        # We will need to read in the atomic masses to compute the molecular weight (mw):
        atomic_masses = pd.read_csv('data/atomicmasses.csv')
        
        # This splits the formula up into elements plus their frequency
        start_i = 0
        elements = []
        for end_i, letter in enumerate(self.formula[1:], start=1):
            if letter.isupper():
                elements.append([start_i, end_i])
                start_i = end_i
        elements.append([start_i, len(self.formula)])

        # This calculates the molecular weight of each type of element
        self.mw = 0
        for element in elements:
            form = self.formula[element[0]:element[1]]
            for i, letter in enumerate(form):
                if letter.isdigit():
                    element_mw = np.float(atomic_masses[atomic_masses.atom == form[:i]]['mass'])
                    self.mw = self.mw + element_mw * np.int(form[i:])
                    break
                if form.isalpha():
                    element_mw = np.float(atomic_masses[atomic_masses.atom == form]['mass'])
                    self.mw = self.mw + element_mw
                    break
        return self.mw
        
    def convert_units(self):
        # This will convert the pressure and adsorption to 'bar' and 'mmol/g', respectively, if needed
        # Pressure Units:
        assert self.pressureUnits == 'bar'

        # Adsorption Units:
        assert self.adsorptionUnits in ['cm3(STP)/g','g/g','mg/g','ml/g','mmol/g','mmol/kg','mol/g','wt%']

        aConversion = 0.0
        if self.adsorptionUnits == 'mmol/g':
            aConversion = 1.0
        elif self.adsorptionUnits == 'mmol/kg':
            aConversion = 1.0 / 1000.0
        elif self.adsorptionUnits == 'mol/g':
            aConversion = 1000.0
        elif self.adsorptionUnits in ['wt%', 'g/g']:
            mw = self.get_molecular_weight()
            aConversion = 1000.0 / mw
        elif self.adsorptionUnits in ['ml/g', 'cm3(STP)/g']:
            aConversion = 1.0 / 22.4
        elif self.adsorptionUnits == 'mg/g':
            mw = self.get_molecular_weight()
            aConversion = 1.0 / mw
        else:
            raise Exception('Something went horribly wrong!')

        self.adsorption = self.adsorption * aConversion
        self.adsorptionUnits = 'mmol/g'
        
    def fit_henry_constant(self, max_adjrms, verbose):
        # This will fit a Henry constant to the adsorption isotherm.
        # `max_adjrms` is a parameter used for the fitting procedure. 
        #  The default value is 0.01, but we can play around with this.
        # `verbose=True` will create a plot of the fit and give additional information
        # If the adsorption units are not in 'mmol/g' and the pressure units are not in 'bar',
        #  the assertion below will throw an error. Use `convert_units` to switch to the correct units
        assert self.adsorptionUnits == 'mmol/g' and self.pressureUnits == 'bar'
        isotherm = pygaps.PointIsotherm(
            pressure=self.pressure,
            loading=self.adsorption,
            material_name=self.adsorbent,
            material_batch='',
            t_iso=self.temperature,
            adsorbate=self.adsorbate)
        try:
            self.henry_constant = pygaps.initial_henry_slope(isotherm, max_adjrms, logx=False, verbose=verbose)
            return self.henry_constant
        except Exception as e:
            self.error = e
            print('Fitting procedure failed! See `self.error` as to why...')
            
    def flag_isotherm(self, henry_df):
        henry_df['ok'].loc[henry_df['filename'] == self.filename] = False
        return
    
    def unflag_isotherm(self, henry_df):
        henry_df['ok'].loc[henry_df['filename'] == self.filename] = True
        return
    
    def add_comment(self, comment, henry_df):
        henry_df['comment'].loc[henry_df['filename'] == self.filename] = comment
        return
    
    def plot_isotherm(self):
        fig = plt.figure(figsize=(8,6))
        bad_datapoints = self.pressure[self.pressure < 0]
        good_datapoints = self.pressure[self.pressure >= 0]
        plt.scatter(self.pressure, self.adsorption, marker='o', label='Experimental')
        if self.henry_constant:
            plt.plot([0, self.pressure[-1]], [0, self.pressure[-1] * self.henry_constant], color='red', label="Henry's Law")
        plt.xlabel('Pressure [{}]'.format(self.pressureUnits))
        plt.ylabel('Adsorption [{}]'.format(self.adsorptionUnits))
        plt.ylim([-0.2*self.adsorption[-1], self.adsorption[-1] * 1.2])
        plt.legend()
        plt.show()
        return
        

In [26]:
henry_df = pd.read_csv('henry_df.csv')
henry_df.head(10)

,Unnamed: 0,gas,material,henry_constant,filename,ok,z_score,comment
0,0,CURLTUGMZLYLDI-UHFFFAOYSA-N,NIST-MATDB-b2dc02871a66ac38e4b195796cf3e00b,0.466816,10.8888s10450-012-9407-1.Isotherm11,True,0.060536,NaN
1,1,CURLTUGMZLYLDI-UHFFFAOYSA-N,NIST-MATDB-b2dc02871a66ac38e4b195796cf3e00b,0.953870,10.8888s10450-012-9407-1.Isotherm1,True,0.060520,NaN
2,2,CURLTUGMZLYLDI-UHFFFAOYSA-N,NIST-MATDB-b2dc02871a66ac38e4b195796cf3e00b,0.290830,10.8888jp206959k.Isotherm17,True,0.060542,NaN
3,3,CURLTUGMZLYLDI-UHFFFAOYSA-N,NIST-MATDB-b2dc02871a66ac38e4b195796cf3e00b,2.612494,10.8888Ja909263x.Isotherm2,True,0.060467,NaN
4,4,CURLTUGMZLYLDI-UHFFFAOYSA-N,NIST-MATDB-b2dc02871a66ac38e4b195796cf3e00b,0.962891,10.8888j.cej.2011.01.096.isotherm4b,True,0.060520,NaN
5,5,CURLTUGMZLYLDI-UHFFFAOYSA-N,NIST-MATDB-b2dc02871a66ac38e4b195796cf3e00b,2.661777,10.1039C4ee01009d.Isotherm22,True,0.060465,NaN
6,6,CURLTUGMZLYLDI-UHFFFAOYSA-N,NIST-MATDB-b2dc02871a66ac38e4b195796cf3e00b,2.136735,10.1039C4ee01009d.Isotherm21,True,0.060482,NaN
7,7,CURLTUGMZLYLDI-UHFFFAOYSA-N,NIST-MATDB-b2dc02871a66ac38e4b195796cf3e00b,2.434755,10.1039C4ee01009d.Isotherm20,True,0.060473,NaN
8,8,CURLTUGMZLYLDI-UHFFFAOYSA-N,NIST-MATDB-b2dc02871a66ac38e4b195796cf3e00b,1.297005,10.1039C4ee01009d.Isotherm19,True,0.060509,NaN
9,9,CURLTUGMZLYLDI-UHFFFAOYSA-N,NIST-MATDB-b2dc02871a66ac38e4b195796cf3e00b,1.263696,10.1039C4ee01009d.Isotherm18,True,0.060510,NaN
